In [59]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

In [60]:
import requests
import tempfile

data_url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/d72b9558a11523adbe13300b41321ecd93d331d3/document.txt"

response = requests.get(data_url)

with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as temp_file:
    temp_file.write(response.content)
    temp_file_path = temp_file.name
    
print(temp_file_path)


/var/folders/77/_452rk390qnf00dq_f0hpvnh0000gn/T/tmpz4uesg3g.txt


In [72]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True
)

loader = UnstructuredFileLoader(file_path=temp_file_path)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = ({
            "context": retriever, 
            "question": RunnablePassthrough(),
        } 
         | prompt 
         | llm
)

def get_cache_data(question):
    loaded_data = load_memory(_)
    print(loaded_data)
    for i in range(0, len(loaded_data), 2):
        human = loaded_data[i]
        if human.content == question:
            hit = loaded_data[i + 1]
            print("memory history hit !!!")
            return hit 
    return None
    
def invoke_chain(question):
    result = get_cache_data(question)
    if result is None:
        result = chain.invoke(question)
        memory.save_context(
            {"input": question},
            {"output": result.content},
        )
    print("[Answer]", result)

invoke_chain("Is Aaronson guilty?")

[]
[Answer] content='According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it suggests that Aaronson is not guilty, but the Party has declared him guilty.'


In [73]:
invoke_chain("Is Aaronson guilty?")

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it suggests that Aaronson is not guilty, but the Party has declared him guilty.')]
memory history hit !!!
[Answer] content='According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it suggests that Aaronson is not guilty, but the Party has declared him guilty.'


In [74]:
invoke_chain("What message did he write in the table?")

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it suggests that Aaronson is not guilty, but the Party has declared him guilty.')]
[Answer] content='He traced with his finger in the dust on the table: 2+2=5.'


In [75]:
invoke_chain("Who is Julia?")

[HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='According to the context, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, but he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it suggests that Aaronson is not guilty, but the Party has declared him guilty.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='He traced with his finger in the dust on the table: 2+2=5.')]
[Answer] content="Julia is a character who is significant to the protagonist, Winston. She is someone he loves and has a deep emotional connection with, as indicated by Winston's overwhelming feelings for her during moments of distress. Julia represents a form of rebellion against the oppressive regime they live under."
